In [1]:
%load_ext autoreload
%autoreload 2

In [20]:
from bn import InPlaceABNSync

ModuleNotFoundError: No module named 'bn'

In [2]:
import os
import wandb
wandb.init(project="sky-eye")
conf = wandb.config

In [3]:
from glob import glob
train_dir = '/home/jupyter/datasets/xview/train'
test_dir = '/home/jupyter/datasets/xview/test'

In [4]:
from tqdm import tqdm_notebook as tqdm
import numpy as np
import torch
from torch import nn

In [5]:
conf.aug_prob = .5

In [6]:
import albumentations as al

augment = al.Compose([
        al.HorizontalFlip(p=conf.aug_prob),
        al.VerticalFlip(p=conf.aug_prob),
        al.RandomRotate90(p=conf.aug_prob),
        al.Transpose(p=conf.aug_prob),
        al.GridDistortion(p=conf.aug_prob, distort_limit=.2),
        al.ShiftScaleRotate(p=conf.aug_prob),
        al.RandomBrightnessContrast(p=conf.aug_prob)
])

In [7]:
conf.batch_size = 12
conf.image_size = 512

In [8]:
from xv.nn import dataset
from xv import util
import random

instances = dataset.XViewSegmentationDataset.get_instances(train_dir)

random.seed(hash("😂"))
random.shuffle(instances)

dev_ix = int(len(instances)*.15)
dev_instances = instances[:dev_ix]
train_instances = instances[dev_ix:]
len(train_instances), len(dev_instances)

train_dataset = dataset.XViewSegmentationDataset(instances=train_instances, resolution=(conf.image_size, conf.image_size), augment=augment)
dev_dataset = dataset.XViewSegmentationDataset(instances=dev_instances, resolution=(conf.image_size, conf.image_size), augment=None)

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=conf.batch_size,
    shuffle=True,
    num_workers=10,
    pin_memory=True,
)

dev_loader = torch.utils.data.DataLoader(
    dev_dataset,
    batch_size=conf.batch_size,
    shuffle=True,
    num_workers=10,
    pin_memory=True,
)

In [9]:
from xv.nn.losses import loss_dict, WeightedLoss

conf.loss_weights = {
    'dice': 1,
    'focal': 1,
    #'bcewithlogits': 1,
    #'jaccard': 1
}

loss = WeightedLoss({loss_dict[l]():w for l, w in conf.loss_weights.items()})

In [10]:
conf.epochs = 100
#conf.pretrained_model = "xdxd_spacenet4"
conf.pretrained_model = 'selimsef_spacenet4_densenet161unet'
conf.pretrained = True
# xdxd_spacenet4

In [11]:
from xv.nn.solaris.model_io import get_model
model = get_model(conf.pretrained_model, 'torch', pretrained=conf.pretrained).to('cuda')

/home/jupyter/sky-eye/xv/nn/solaris/zoo/selim_sef_sn4.py:474: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


In [12]:
import apex

optims = {
    'adam': torch.optim.Adam,
    #'adamApex': apex.optimizers.FusedAdam,
    'adamw': torch.optim.AdamW
}

conf.optim = 'adam'
conf.lr = 0.0005

optim = optims[conf.optim](model.parameters(), lr=conf.lr)

In [13]:
from apex import amp
conf.amp_opt_level = 'O1'
model, optim = amp.initialize(model, optim, opt_level=conf.amp_opt_level)

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


In [ ]:
wandb.watch(model)

In [14]:
conf.scheduler_factor = 0.5
conf.scheduler_patience = 5

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optim, factor=conf.scheduler_factor, patience=conf.scheduler_patience)

In [15]:
from torchvision.models.resnet import BasicBlock, Bottleneck

In [16]:
from collections import defaultdict
def train(model, optim, data, loss_fn):
    model.train()
    loss_sum = 0.
    
    for batch in tqdm(iter(data)):
        optim.zero_grad()
        loss_buildings = loss_fn(model(batch['images']['image'].to('cuda')),
                                 batch['masks']['buildings'].to('cuda'))
        loss_sum += loss_buildings
        
        #loss_buildings.backward()
        
        with amp.scale_loss(loss_buildings, optim) as scaled_loss:
            scaled_loss.backward()
            
        optim.step()
        
        #loss_damage = loss_fn(model(batch['images']['post'].to('cuda')),
        #                   batch['masks']['damage'])
        
        #with amp.scale_loss(loss_damage, optim) as scaled_loss:
        #    scaled_loss.backward()
        
        #optim.zero_grad()
    return loss_sum / len(data)
        
def evaluate(model, optim, data, loss_fn, threshold=0.5):
    model.eval()
    metrics = defaultdict(float)
    
    with torch.no_grad():
        for batch in tqdm(iter(data)):
            outputs = model(batch['images']['image'].to('cuda'))
            targets = batch['masks']['buildings'].to('cuda')
            metrics['loss'] += loss_fn(outputs, targets)
            
            pr_sum, re_sum, f_sum = 0., 0., 0.
            for output, target in zip(outputs, targets):
                target_bool = target.bool()
                output_bool = output.sigmoid() > threshold

                recall = output_bool[target_bool].float().mean()
                recall = recall if recall == recall else 1.

                precision = target_bool[output_bool].float().mean()
                precision = precision if precision == precision else 1.
                
                pr_sum += precision
                re_sum += recall
                f_sum += 2*precision*recall/(precision + recall) if precision + recall > 0. else 0.
            
            metrics['recall'] += pr_sum/len(outputs)
            metrics['precision'] += re_sum/len(outputs)
            metrics['f1'] += f_sum/len(outputs)
            
            
    return {k:v/len(data) for k, v in metrics.items()}

In [17]:
EPOCHS = 100
epoch = 0

In [ ]:
for epoch in range(epoch, EPOCHS):
    metrics = {}
    metrics['train_loss'] = train(model, optim, train_loader, loss)
    metrics.update({f'dev_{k}':v for k,v in evaluate(model, optim, dev_loader, loss).items()})
    scheduler.step(metrics['dev_loss'])
    wandb.log(metrics)

Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0


In [ ]:
wandb.log(dict(train_loss=train_loss, eval_loss=eval_loss))

In [ ]:
from torchvision.models.resnet import BasicBlock

In [ ]:
class BuildingDamageHeatmap(nn.Module):
    def __init__(self, segmentation):
        self.segmentation = segmentation
        

In [ ]:
ix = 1000
i = train_dataset[ix]
images, masks = i['images'], i['masks']
image = images['post']
image = np.array(train_dataset.inverse_transform_image(image))

util.vis_im_mask(image, masks['damage'], size=(512*2,512*2), opacity=.3);

In [ ]:
from collections import Counter
counts = Counter(len(i['pre']['features']) for i in train_dataset.instances)

In [ ]:
counts[0]/sum(counts.values())